In [1]:
from roboflow import Roboflow
import cv2
import numpy as np
import supervision as sv

In [28]:
rf = Roboflow(api_key="EQazb1etxBV71iH6TAIX")
project = rf.workspace('kshi2').project("chelsea_newcastle_test")
model = project.version(1312).model

loading Roboflow workspace...
loading Roboflow project...


In [29]:
cd c:/project/yolo_football_video_MoT

c:\project\yolo_football_video_MoT


In [32]:
dataset=project.version(1312).download('yolov9')


Extracting Dataset Version Zip to chelsea_newcastle_test-1312 in yolov9:: 100%|██████████| 216/216 [00:00<00:00, 1148.69it/s]


In [7]:
mp4='C:/project/yolo_football_video_MoT/data/chelsea_newcastle_test.mp4'

In [8]:
video_info=sv.VideoInfo.from_video_path(mp4)

In [9]:
video_info

VideoInfo(width=1920, height=1080, fps=30, total_frames=356)

In [13]:
job_id, signed_url, expire_time = model.predict_video(
    mp4,
    fps=5,
    prediction_type="batch-video"
)

results = model.poll_until_video_results(job_id)

print(results)

TypeError: InferenceModel.predict_video() got an unexpected keyword argument 'save'

In [12]:
results.keys()

dict_keys(['frame_offset', 'time_offset', 'chelsea_newcastle_test'])

In [26]:
results['chelsea_newcastle_test'][58]['time']

0.17625429388135672

In [40]:
dataset.location

'c:\\project\\yolo_football_video_MoT\\chelsea_newcastle_test-1312'

In [35]:
yolo task=detect mode=train model=yolov9.pt data={dataset.location}/data.yaml epochs=100 imgsz=640

SyntaxError: invalid syntax (1385235016.py, line 1)

In [1]:
from ultralytics import YOLO

In [2]:
model=YOLO('yolov9e')

  1%|          | 792k/112M [00:07<28:31, 68.2kB/s] 

In [11]:
# 비디오 쓰기 객체 생성
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (640, 480))

In [27]:
def callback(scene: np.ndarray, index: int) -> np.ndarray:
    results = model.poll_until_video_results(job_id)[index]
    detections = sv.Detections.from_ultralytics(results)

    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()

    labels = [
        results.names[class_id]
        for class_id
        in detections.class_id
    ]

    annotated_image = bounding_box_annotator.annotate(
        scene=scene, detections=detections)

    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections, labels=labels)

    return annotated_image

sv.process_video(
    source_path=mp4,
    target_path="output.mp4",
    callback=callback
)

Checking for video inference results for job f2bf7b3f-a4f7-482e-85ff-b96bd7a6bf05 every 60s
(0s): Checking for inference results


KeyError: 0

In [22]:
mp4read=cv2.VideoCapture(mp4)
cap,frame=mp4read.read()

In [29]:
frame[:,:,:].shape

(1080, 1920, 3)

In [7]:
# Hex 색상코드를 RGB색상코드로 변경
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

In [11]:
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        # 프레임에서 객체 감지
        prediction = model.predict(frame, confidence=50)
        
        # 결과를 프레임에 표시
        for result in prediction:
            bottom_right = (round(result['x']-result['width']/2), round(result['y']-result['height']/2))
            top_left = (round(result['x']+result['width']/2), round(result['y']+result['height']/2))
            color = hex_to_rgb(version.colors[result['class']])
            thickness = 2
            cv2.rectangle(frame, bottom_right, top_left, color, thickness)
            cv2.putText(frame, result['class'], (round(result['x']-result['width']/2), round(result['y']-result['height']/2-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color=color, thickness=3)
        
        # 결과 프레임을 파일로 저장
        out.write(frame)
    else:
        break

# 모든 작업이 완료되면 캡처와 쓰기 객체를 해제
cap.release()
out.release()

HTTPError: 403 Client Error: Forbidden for url: https://detect.roboflow.com/chelsea_newcastle/1?api_key=7LaCX25FUT6blBXw40FE&name=YOUR_IMAGE.jpg&overlap=30&confidence=50&stroke=1&labels=false&format=json